Finetuning Technique: In this method we adjust the abstract level features and freeze the initial layers due to generic features. Adjust the top layers along with the top classifier layers. 

* In this example can fine-tune the conv5 of VGG16 along with the fully connected top layers.

Steps of fine-tunning are:
* Add your custom network on top of an already-trained base network
* Freeze the base network
* Train the part you added 
* Unfreeze some layers in base network 
* Jointly train both these layers and the part you added 

In [1]:
from keras.applications import VGG16

In [2]:
conv_base = VGG16(weights='imagenet',include_top = False, input_shape= (150,150,3))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

We will now froze the conv upto block4 and train block5

The two reasons to freeze the layers below the conv5 are:
    * The layers below extarcts more generic features that can be appliable to other problem 
    * The number are features that below layers have in million so if we train the model on a small dataset that can lead to overfitting 

So, a good strategy is to train the top layers (including conv5 and the fully connected layers)

In [9]:
conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
    
    if layer.name =='block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

make the learning rate smaller to make the learning smooth (limits the magnitude of modification) 

Start fine-tuning the Model

In [12]:
from keras import optimizers
from keras import models
from keras import layers 

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-5),loss='binary_crossentropy',metrics=['acc'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


In [19]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import os

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range= 40,
                                  width_shift_range= 0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
                                 

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [20]:
base_dir = 'C:/Users/Haier/Dataset/dogs-vs-cats/cats_and_dogs_small'
train_dir = os.path.join(base_dir,'train').replace('\\','/')

In [21]:
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150,150),
                                                   batch_size=20, class_mode='binary')

Found 2000 images belonging to 2 classes.


In [22]:
validation_dir = os.path.join(base_dir,'validation').replace('\\','/')

test_dir = os.path.join(base_dir,'test').replace('\\','/')

In [23]:
validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(150,150),
                                                       batch_size=20,
                                                       class_mode='binary')

Found 1000 images belonging to 2 classes.


In [24]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=100,
                             epochs=5,
                             validation_data=validation_generator,
                             validation_steps=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
100/100 [==============================] - 811s 8s/step - loss: 0.5580 - acc: 0.6985 - val_loss: 0.3334 - val_acc: 0.8600
Epoch 2/5
100/100 [==============================] - 812s 8s/step - loss: 0.3810 - acc: 0.8200 - val_loss: 0.2451 - val_acc: 0.8940
Epoch 3/5
100/100 [==============================] - 813s 8s/step - loss: 0.3327 - acc: 0.8555 - val_loss: 0.2186 - val_acc: 0.9010
Epoch 4/5
100/100 [==============================] - 851s 9s/step - loss: 0.2928 - acc: 0.8700 - val_loss: 0.2049 - val_acc: 0.9100
Epoch 5/5
100/100 [==============================] - 884s 9s/step - loss: 0.2701 - acc: 0.8925 - val_loss: 0.2094 - val_acc: 0.9090


In [25]:
model.save('cats_vs_dogs_small_dataset_finetuning_model3.h5')